# HDS5210-2024 Midterm

In the midterm, you're going to use all the programming and data management skills you've developed so far to build a risk calculator that pretends to be integrated with a clinical registry.  You'll compute the PRIEST COVID-19 Clinical Severity Score for a series of patients and, based on their risk of an adverse outcome, query a REST web service to find a hospital to transfer them to. The end result of your work will be a list of instructions on where each patient should be discharged given their risk and various characteristics of the patient.

Each step in the midterm will build up to form your complete solution.

**Make sure you write good docstrings and doctests along the way!!**

**The midterm is due at 11:59 PM CST on Monday, October 24th.**

---

## Step 1: Calculate PRIEST Clinical Severity Score

This scoring algorithm can be found [here on the MDCalc website](https://www.mdcalc.com/priest-covid-19-clinical-severity-score#evidence).  

1. You will need to write a function called **priest()** with the following input parameters.  
 * Sex (Gender assigned at birth)
 * Age in years
 * Respiratory rate in breaths per minute
 * Oxygen saturation as a percent between 0 and 1
 * Heart rate in beats per minute
 * Systolic BP in mmHg
 * Temperature in degrees C
 * Alertness as a string description
 * Inspired Oxygen as as string description
 * Performance Status as a string description
2. The function will need to follow the algorithm provided on the MDCalc website to compute a risk percentage that should be returned as a numeric value between 0 and 1.
3. Be sure to use docstring documentation and at least three built-in docstring test cases.
4. Assume that the input values that are strings could be any combination of upper or lower case. For example: 'male', 'Male', 'MALE', 'MalE' should all be interpretted by your code as male.
5. If any of the inputs are invalid (for example a sex value that is not recognizable as male or female) your code should return None.

NOTES:
1. In the final step there is a table that translates from **PRIEST Score** to **30-day probability of an outcome** but the last two probabilities are shown as ranges (59-88% and >99%).  Our code needs to output a single number, however. For our code, use the following rule:
 * If PRIEST score is between 17 and 25, the probability you return should be 0.59
 * If PRIEST score is greater than or equal to 26, the probability you return should be 0.99


In [24]:
def priest(gender, patient_age, breaths_per_minute, spO2, pulse, blood_pressure, body_temp,
                           consciousness, oxygen_inhalation, mobility):
    """
    (string, int, float, float, float, float, float, string, string, string) -> float

    Calculates the PRIEST score to assess risk of severe outcomes from respiratory infections.
    This function considers patient details like age, vital signs, and physical status to compute a risk percentage.
    It helps in triaging patients during pandemics based on their medical condition and potential for adverse outcomes.

    Args:
    gender (str): 'male' or 'female'.
    patient_age (int): Age in years.
    breaths_per_minute (int): Respiratory rate.
    spO2 (float): Oxygen saturation, 0 to 1 scale.
    pulse (int): Heart rate in beats per minute.
    blood_pressure (int): Systolic blood pressure in mmHg.
    body_temp (float): Body temperature in Celsius.
    consciousness (str): 'alert' or 'confused/not alert'.
    oxygen_inhalation (str): 'air' or 'supplemental oxygen'.
    mobility (str): Patient's mobility and activity level.

    Returns:
    float: Probability of a severe outcome within 30 days.

    Examples:
    >>> calculate_priest_score('female', 80, 31, 0.83, 60, 91, 35.0, 'confused/not alert', 'supplemental oxygen', 'bed/chair bound, no self-care')
    0.59

    >>> calculate_priest_score('male', 50, 20, 0.53, 90, 102, 19.0, 'alert', 'air', 'unrestricted normal activity')
    0.29
    """
    score = 0

    # Gender scoring
    if gender.lower() == 'male':
        score += 1

    # Age scoring
    if 50 <= patient_age <= 65:
        score += 2
    elif 66 <= patient_age <= 80:
        score += 3
    elif patient_age > 80:
        score += 4

    # Respiratory rate scoring
    if breaths_per_minute < 9:
        score += 3
    elif 9 <= breaths_per_minute <= 11:
        score += 1
    elif 12 <= breaths_per_minute <= 20:
        score += 0
    elif 21 <= breaths_per_minute <= 24:
        score += 2
    elif breaths_per_minute > 24:
        score += 3

    # Oxygen saturation scoring
    if spO2 <= 0.91:
        score += 3
    elif 0.92 <= spO2 <= 0.93:
        score += 2
    elif 0.94 <= spO2 <= 0.95:
        score += 1

    # Heart rate scoring
    if pulse < 41:
        score += 3
    elif 41 <= pulse <= 50:
        score += 1
    elif 51 <= pulse <= 90:
        score += 0
    elif 91 <= pulse <= 110:
        score += 1
    elif 111 <= pulse <= 130:
        score += 2
    elif pulse > 130:
        score += 3

    # Blood pressure scoring
    if blood_pressure < 91:
        score += 3
    elif 91 <= blood_pressure <= 100:
        score += 2
    elif 101 <= blood_pressure <= 110:
        score += 1

    # Temperature scoring
    if body_temp < 35.1:
        score += 3
    elif 35.1 <= body_temp <= 36.0:
        score += 1
    elif 36.1 <= body_temp <= 38.0:
        score += 0
    elif 38.1 <= body_temp <= 39.0:
        score += 1
    elif body_temp > 39.0:
        score += 2

    # Consciousness scoring
    if consciousness.lower() != 'alert':
        score += 3

    # Oxygen inhalation scoring
    if oxygen_inhalation.lower() == 'supplemental oxygen':
        score += 2

    # Mobility scoring
    mobility_dict = {
        'unrestricted normal activity': 0,
        'limited strenuous activity, can do light activity': 1,
        'limited activity, can self-care': 2,
        'limited self-care': 3,
        'bed/chair bound, no self-care': 4
    }
    score += mobility_dict.get(mobility.lower(), 0)

    # Risk probabilities based on total score
    risk_mapping = {
        1: 0.01, 2: 0.02, 3: 0.03, 4: 0.03, 5: 0.09, 6: 0.15, 7: 0.18, 8: 0.22, 9: 0.26, 10: 0.29,
        11: 0.34, 12: 0.38, 13: 0.46, 14: 0.47, 15: 0.49, 16: 0.59
    }
    return risk_mapping.get(score, 0.59 if score >= 16 else 0.99)


In [25]:
import doctest
doctest.run_docstring_examples(priest, globals(),verbose=True)

Finding tests in NoName
Trying:
    calculate_priest_score('female', 80, 31, 0.83, 60, 91, 35.0, 'confused/not alert', 'supplemental oxygen', 'bed/chair bound, no self-care')
Expecting:
    0.59
**********************************************************************
File "__main__", line 26, in NoName
Failed example:
    calculate_priest_score('female', 80, 31, 0.83, 60, 91, 35.0, 'confused/not alert', 'supplemental oxygen', 'bed/chair bound, no self-care')
Exception raised:
    Traceback (most recent call last):
      File "/usr/lib/python3.10/doctest.py", line 1350, in __run
        exec(compile(example.source, filename, "single",
      File "<doctest NoName[0]>", line 1, in <module>
        calculate_priest_score('female', 80, 31, 0.83, 60, 91, 35.0, 'confused/not alert', 'supplemental oxygen', 'bed/chair bound, no self-care')
    NameError: name 'calculate_priest_score' is not defined
Trying:
    calculate_priest_score('male', 50, 20, 0.53, 90, 102, 19.0, 'alert', 'air', 'unrestrict

## Part 2: Find a hospital

The next thing we have to do is figure out where to send this particular patient.  The guidelines on where to send a patient are based on their age (pediatric, adult, geriatric), sex, and risk percentage.  Luckily, you don't have to implement these rules. I already have. All you have to do is use a REST web service that I've created for you.

You'll want to use Python to make a call to my REST web service similar to the example URL below. The first part of the URL will be the same for everyone and every request that you make. What you will need to modify for each of your requests is the information after the question mark.

```
https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/?age=40&sex=male&risk_pct=0.1
```

The example above asks my web service where a 40-year old male with a risk of 10% should go.  What the web service will return back is a JSON string containing the information you need.  That JSON will look like this:

```json
{
  "age": "40",
  "sex": "male",
  "risk": "0.1",
  "hospital": "Southwest Hospital and Medical Center"
}
```

My function is not smart enough to understand `'MALE'` is the same as `'male'`.  You have to send it exactly `'male'` or `'female'`

1. Your job is to write a function called **find_hospital()** that takes age, sex, and risk as parameters.
2. Your function should call this REST web service using the `requests` module
3. Then your function will need to interpret the JSON it gets and return just the name of the hospital
4. If anything fails, return None
5. Include a good docstring with at least three test cases.


In [26]:
pip install requests

In [27]:
import requests

def find_hospital(age, sex, risk_pct):
    """
    (int, string, float) -> String
    Fetches the recommended hospital from a REST API based on patient details.
    This function communicates with an API to find a hospital suitable for a patient,
    considering their age, gender, and risk percentage of severe outcomes from a health condition.

    Inputs:
    age (int): The age of the patient.
    sex (str): The sex of the patient ('male' or 'female').
    risk_pct (float): The risk percentage of the patient as a decimal.

    Output:
    str: The name of the recommended hospital, or None if an error occurs.

    Examples:
    >>> find_hospital(80, 'female', 0.4)
    'Wesley Woods Geriatric Hospital'

    >>> find_hospital(83, 'female', 0.9)
    'Wesley Woods Geriatric Hospital'

    >>> find_hospital(76, 'female', 0.1)
    'Wesley Woods Geriatric Hospital'

    >>> find_hospital(30, 'male', 0.5)
    'Emory Dunwoody Medical Center'
    """
    # URL for the REST API endpoint
    base_url = "https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/"
    # Parameters to be sent in the query string
    params = {'age': age, 'sex': sex, 'risk_pct': risk_pct}

    try:
        # Making the GET request to the API
        response = requests.get(base_url, params=params)
        # Check if the request was successful
        if response.status_code == 200:
            # Parsing the JSON data from the response
            data = response.json()
            # Returning the hospital name
            return data.get('hospital')
        else:
            # Handle situations where the API does not return a 200 OK
            return None
    except Exception as e:
        # Handle any exceptions that occur during the request
        print(f"An error occurred: {e}")
        return None


In [28]:
import doctest
doctest.run_docstring_examples(find_hospital, globals(),verbose=True)

Finding tests in NoName
Trying:
    find_hospital(80, 'female', 0.4)
Expecting:
    'Wesley Woods Geriatric Hospital'
ok
Trying:
    find_hospital(83, 'female', 0.9)
Expecting:
    'Wesley Woods Geriatric Hospital'
ok
Trying:
    find_hospital(76, 'female', 0.1)
Expecting:
    'Wesley Woods Geriatric Hospital'
ok
Trying:
    find_hospital(30, 'male', 0.5)
Expecting:
    'Emory Dunwoody Medical Center'
ok


## Part 3: Get the address for that hospital from a JSON file

Great! Now we have code to tell us which hospital to send someone to... but we don't know where that hospital is. The next function we need to create is one that looks up the address of that hospital.  All of these hospitals are in Atlanta, Georgia.  We're going to use the list from this webpage to lookup the address for that hospital, based on its name.  https://www.officialusa.com/stateguides/health/hospitals/georgia.html

Because we skipped the section about Beautiful Soup and working with HTML, I've converted this information into a JSON document for you.  It's available for you here.  Your code should retrieve this file using the `requests` module.

`https://drive.google.com/uc?export=download&id=1fIFD-NkcdiMu941N4GjyMDWxiKsFJBw-`

1. You need to create a function called **get_address()** that takes hospital name as a parameter and searches the data from this JSON file for the hospital you want to find.
2. Your code will have to load the JSON and return the correct hospital based on name.
3. If the hospital name isn't found, the function should return None.
4. Be sure to use good docstring documentation and includes at least 3 test cases.

In [29]:
import requests

def get_address(hospital_name):
    """
    (String) -> String
    Looks up the address of a specific hospital using its name from a JSON file online.
    This function fetches and parses JSON data from a specified URL to locate the address of a hospital.
    It requires the exact hospital name to search the dataset and returns the corresponding address.

    Args:
    hospital_name (str): The name of the hospital to find the address for.

    Returns:
    str: The address of the hospital, or None if the hospital is not found in the JSON data.

    Examples:
    >>> get_address("HEALTHSOUTH REHABILITATION HOSPITAL OF NEWNAN")
    '2101 EAST NEWNAN CROSSING BLVD'

    >>> get_address("PERRY HOSPITAL")
    '1120 MORNINGSIDE DRIVE'

    >>> get_address("STEWART WEBSTER HOSPITAL")
    '580 ALSTON STREET'
    """
    # URL of the JSON file containing hospital addresses
    json_url = "https://drive.google.com/uc?export=download&id=1fIFD-NkcdiMu941N4GjyMDWxiKsFJBw-"

    try:
        # Making the GET request to download the JSON data
        response = requests.get(json_url)
        # Check if the request was successful
        if response.status_code == 200:
            # Parsing the JSON data
            hospitals = response.json()
            # Normalize the hospital name for key matching
            key = hospital_name.upper()
            if key in hospitals:
                # Retrieve the hospital data
                hospital_data = hospitals[key]
                # Construct the address string
                address = f"{hospital_data['ADDRESS']}"
                return address
            # If no match is found
            return None
        else:
            # Handle situations where the JSON data could not be retrieved
            return None
    except Exception as e:
        # Handle any exceptions that occur during the request
        print(f"An error occurred: {e}")
        return None



In [30]:
doctest.run_docstring_examples(get_address, globals(),verbose=True)

Finding tests in NoName
Trying:
    get_address("HEALTHSOUTH REHABILITATION HOSPITAL OF NEWNAN")
Expecting:
    '2101 EAST NEWNAN CROSSING BLVD'
ok
Trying:
    get_address("PERRY HOSPITAL")
Expecting:
    '1120 MORNINGSIDE DRIVE'
ok
Trying:
    get_address("STEWART WEBSTER HOSPITAL")
Expecting:
    '580 ALSTON STREET'
ok


## Part 4: Run the risk calculator on a population

At the link below, there is a file called `people.psv`.  It is a pipe-delimited (`|`) file with columns that match the inputs for the PRIEST calculation above.  Your code should use the `requests` module to retrieve the file from this URL.

`https://drive.google.com/uc?export=download&id=1fLxJN9YGUqmqExrilxSS8furwUER5HHh`


In addition, the file has a patient identifier in the first column.

1. Write a function called **process_people()** that takes the file location above as its only parameter. Your Python program should use your code above to process all of these rows, determine the hospital and address, and return a list whose items are a dictionary like this: `{ patient_number: [sex, age, breath, o2sat, heart, systolic, temp, alertness, inspired, status, hospital, address]}`.  Look at the file in Part 5 for what the output looks like.
2. Be sure to use good docstrings, but you don't need any tests in your doc strings.  I've provided those for you withe file in Part 5.


**NOTE** that when running your code for all the 100 records in the `people.psv` file, it may take a few minutes to complete.  You're making multiple calls to the internet for each record, so that can take a little while.


In [31]:
def process_people(file_url):
    """
    Processes a pipe-delimited file of patient data to calculate their PRIEST score, determine the appropriate
    hospital, and retrieve the hospital address.

    Args:
    file_url (str): The URL where the patient data file is stored.

    Returns:
    list: A list of dictionaries, each containing patient information and calculated results.
    """
    results = []
    try:
        # Retrieve the file from the URL
        response = requests.get(file_url)
        # Ensure successful download
        if response.status_code == 200:
            # Split the content into lines
            lines = response.text.splitlines()
            # Iterate over each line, skipping the header
            for line in lines[1:]:
                fields = line.split('|')
                # Extract fields based on the known structure
                patient_number, sex, age, breath, o2sat, heart, systolic, temp, alertness, inspired, status = fields
                # Convert types as necessary
                age = int(age)
                print(fields)
                sex=sex.lower()
                breath = float(breath)
                o2sat = float(o2sat)
                heart = int(heart)
                systolic = float(systolic)
                temp = float(temp)
                alertness = alertness.lower()
                inspired = inspired.lower()
                status = status.lower()
                print(fields)
                # Calculate PRIEST score
                risk_pct = priest(sex, age, breath, o2sat, heart, systolic, temp, alertness, inspired, status)
                print(risk_pct)
                # Find appropriate hospital
                hospital = find_hospital(age, sex, risk_pct)
                # Get the hospital's address
                address = get_address(hospital)

                # Append results in the specified dictionary format
                results.append({
                    patient_number: [
                        sex, age, breath, o2sat, heart, systolic, temp, alertness, inspired, status, hospital, address
                    ]
                })
        else:
            print("Failed to download the file.")
    except Exception as e:
        print(f"An error occurred while processing the data: {e}")

    return results


## Part 5: Checking your final results

The final step is to check your results.  You should be able to compare your results to the output in `people_results.json` at the link below.  Write some code to check your results.  This does not need to be a function.

`https://drive.google.com/uc?export=download&id=1gx1SSC20mO5XL6uYD0mdcM_cL91fcIW5`


In [23]:
file_url = "https://drive.google.com/uc?export=download&id=1fLxJN9YGUqmqExrilxSS8furwUER5HHh"
result = process_people(file_url)

# URL for the expected results
expected_results_url = "https://drive.google.com/uc?export=download&id=1gx1SSC20mO5XL6uYD0mdcM_cL91fcIW5"

response = requests.get(expected_results_url)
expected_results = response.json()

# Convert your results into a dictionary for easier access
your_results_dict = {list(item.keys())[0]: item[list(item.keys())[0]][-2:] for item in result}

# Function to compare expected and actual results
def compare_results(expected, actual):
    discrepancies = []
    for patient_id, data in expected.items():
        expected_hospital_name = data[11]  # Assuming 'data' is a list from the JSON with the correct index for the hospital name
        expected_address = data[12]        # and address.
        actual_hospital_name, actual_address = actual.get(patient_id, ('No Data', 'No Data'))

        if (expected_hospital_name != actual_hospital_name) or (expected_address != actual_address):
            discrepancies.append({
                'Patient ID': patient_id,
                'Expected Hospital': expected_hospital_name,
                'Actual Hospital': actual_hospital_name,
                'Expected Address': expected_address,
                'Actual Address': actual_address
            })
    return discrepancies

# Perform the comparison
discrepancies = compare_results(expected_results, your_results_dict)

# Print out discrepancies
if discrepancies:
    print("Discrepancies found:")
    for discrepancy in discrepancies:
        print(discrepancy)
else:
    print("No discrepancies found. All results match.")

['E9559', 'FEMALE', '40', '24', '0.96', '105', '115', '34.9', 'ALERT', 'AIR', 'unrestricted normal activity']
['E9559', 'FEMALE', '40', '24', '0.96', '105', '115', '34.9', 'ALERT', 'AIR', 'unrestricted normal activity']
0.15
['E9385', 'Female', '51', '19', '0.99', '90', '91', '38.1', 'ALERT', 'supplemental oxygen', 'unrestricted normal activity']
['E9385', 'Female', '51', '19', '0.99', '90', '91', '38.1', 'ALERT', 'supplemental oxygen', 'unrestricted normal activity']
0.18
['E3067', 'female', '40', '29', '0.96', '105', '95', '38.1', 'ALERT', 'air', 'unrestricted normal activity']
['E3067', 'female', '40', '29', '0.96', '105', '95', '38.1', 'ALERT', 'air', 'unrestricted normal activity']
0.18
['E9422', 'FEMALE', '66', '19', '0.96', '135', '115', '38.1', 'ALERT', 'AIR', 'bed/chair bound, no self-care']
['E9422', 'FEMALE', '66', '19', '0.96', '135', '115', '38.1', 'ALERT', 'AIR', 'bed/chair bound, no self-care']
0.34
['E8661', 'MALE', '18', '22', '0.95', '105', '115', '36.3', 'Confused or

---

## Check your work above

If you didn't get them all correct, take a few minutes to think through those that aren't correct.


## Submitting Your Work

Submit your work as usual into a folder named `midterm`

---